In [1]:
import sys
sys.path.append("..")

In [2]:
label_path = '../data/label.pkl'
file = '/Users/knpob/Territory/Kolmo/code/foot-measure/data/41.obj'
is_plot = True

## Load landmarks labelling

In [3]:
import pandas as pd

df = pd.read_pickle(label_path)
df

coord                                                                x  \
file                                               landmark              
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1         3.857714   
                                                   P10       39.322110   
                                                   P11       60.724825   
                                                   P12      -12.828403   
                                                   P2        39.049710   
                                                   P3       -36.862691   
                                                   P4        60.973624   
                                                   P5       -35.013347   
                                                   P6        20.691139   
                                                   P7        21.423839   
                                                   P8        62.182012   
                                                   P9         2.314420   
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1       -17.238714   
                                                   P10       27.044899   
                                                   P11       46.314558   
                                                   P12      -12.425996   
                                                   P2        23.827222   
                                                   P3       -48.912331   
                                                   P4        37.029041   
                                                   P5       -43.374166   
                                                   P6        10.874670   
                                                   P7        15.162151   
                                                   P8        47.552363   
                                                   P9        -3.474880   
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        13.057726   
                                                   P10       24.043601   
                                                   P11       52.286248   
                                                   P12      -15.867874   
                                                   P2        51.554052   
                                                   P3       -42.383780   
                                                   P4        55.427981   
                                                   P5       -41.360362   
                                                   P6        24.159899   
                                                   P7        25.623090   
                                                   P8        49.666306   
                                                   P9       -11.454836   
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        -7.654096   
                                                   P10       28.590385   
                                                   P11       36.689597   
                                                   P12      -25.769309   
                                                   P2        32.462991   
                                                   P3       -52.356440   
                                                   P4        40.296835   
                                                   P5       -52.167985   
                                                   P6         8.157284   
                                                   P7         8.480356   
                                                   P8        46.692287   
                                                   P9        -9.043764   
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1         3.693454   
                                                   P10       28.918712   
                                                   P11       53.428009   
                                                   P12       -8.152524   
    

## Local frame

In [4]:
import pyvista as pv
from measure import frame, visual
from mesh4d.analyse import crave

mesh = crave.fix_pvmesh_disconnect(pv.read(file))
axes_frame, origin = frame.estimate_foot_frame(mesh, file, df)

if is_plot:
    visual.plot_axes(origin, axes_frame, mesh)

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe3508359a0_0&reconnect=auto' style='widt…

In [5]:
mesh_clip = frame.foot_clip(mesh, df, file)
mesh_local = frame.foot2local(mesh_clip, axes_frame, origin)

if is_plot:
    mesh_local.plot()

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe35377df10_1&reconnect=auto' style='widt…

In [6]:
df_local = frame.df2local(df, axes_frame, origin)
df_local

coord                                                                 x  \
file                                               landmark               
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        112.678710   
                                                   P10      -107.682720   
                                                   P11       -34.106212   
                                                   P12       -53.918829   
                                                   P2        104.242322   
                                                   P3         65.908776   
                                                   P4         64.309920   
                                                   P5         34.325621   
                                                   P6         15.770394   
                                                   P7         -1.727357   
                                                   P8        -61.800817   
                                                   P9        -75.515808   
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1       -115.832381   
                                                   P10       109.179147   
                                                   P11        51.399665   
                                                   P12        69.218368   
                                                   P2       -102.120681   
                                                   P3        -60.048257   
                                                   P4        -62.571968   
                                                   P5        -30.425519   
                                                   P6         -2.225621   
                                                   P7         16.918225   
                                                   P8         79.861482   
                                                   P9         83.861775   
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        111.448303   
                                                   P10      -126.667682   
                                                   P11       -51.406494   
                                                   P12       -72.565245   
                                                   P2         95.315839   
                                                   P3         61.116501   
                                                   P4         54.824755   
                                                   P5         25.689752   
                                                   P6         -1.225772   
                                                   P7        -17.561817   
                                                   P8        -87.476169   
                                                   P9        -96.038766   
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        -74.541692   
                                                   P10       153.178491   
                                                   P11       100.122663   
                                                   P12        93.722053   
                                                   P2        -67.091405   
                                                   P3        -24.562393   
                                                   P4        -10.350199   
                                                   P5          5.343338   
                                                   P6         36.030698   
                                                   P7         50.355532   
                                                   P8        119.389118   
                                                   P9        134.252059   
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        -50.754140   
                                                   P10       174.870699   
                                                   P11       122.616014   
                         

## Metrics

In [7]:
results = [
    {
        'file': 'description',
        'FL': 'foot length (mm)',
        'MBL': 'medial ball length (mm)',
        'LBL': 'lateral ball length (mm)',
        'ABW': 'anatomical ball width (mm)',
        'OBW': 'orthogonal ball width (mm)',
        'OHW': 'orthogonal heel width (mm)',
        'BH': 'ball heigh (mm)',
        'IH': 'instep height (mm)',
        'BA': 'ball angle (°)',
        'T1A': 'toe 1 angle (°)',
        'T5A': 'toe 5 angle (°)',
        'ABG': 'anatomical ball girth (mm)',
        'IG': 'instep girth (mm)',
    }
]

In [8]:
from measure import metric

results.append(
    {
        'file': file,
        'FL': metric.fl(df_local, file),
        'MBL': metric.mbl(df_local, file),
        'LBL': metric.lbl(df_local, file),
        'ABW': metric.abw(df_local, file),
        'OBW': metric.obw(df_local, file),
        'OHW': metric.ohw(df_local, file),
        'BH': metric.bh(df_local, file),
        'IH': metric.ih(df_local, file),
        'BA': metric.ba(df_local, file),
        'T1A': metric.t1a(df_local, file),
        'T5A': metric.t5a(df_local, file),
        'ABG': metric.abg(df_local, file, mesh_local),
        'IG': metric.ig(df_local, file, mesh_local),
    }
)

In [9]:
df_metric = pd.DataFrame(results).set_index('file')
df_metric

,FL,MBL,LBL,ABW,OBW,OHW,BH,IH,BA,T1A,T5A,ABG,IG
file,,,,,,,,,,,,,
description,foot length (mm),medial ball length (mm),lateral ball length (mm),anatomical ball width (mm),orthogonal ball width (mm),orthogonal heel width (mm),ball heigh (mm),instep height (mm),ball angle (°),toe 1 angle (°),toe 5 angle (°),anatomical ball girth (mm),instep girth (mm)
/Users/knpob/Territory/Kolmo/code/foot-measure/data/4.obj,238.115985,181.492436,152.357434,102.160496,97.762887,61.445106,61.512783,70.806672,69.207947,10.299569,12.465447,513.590166,237.267636


### Visualization

In [10]:
if is_plot:
    settings = {
        'FL': ['P1', 'P10', 'x'],
        'MBL': ['P4', 'P10', 'x'],
        'LBL': ['P5', 'P10', 'x'],
        'OBW': ['P4', 'P3', 'y'],
        'OHW': ['P9', 'P8', 'y'],
    }

    for name, [landmark1, landmark2, axis] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_dist_along_axis(scene, df_local, file, landmark1, landmark2, axis, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe371ccf940_2&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe3394a8e80_3&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe3394ca670_4&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe3394ac6d0_5&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe3540f5850_6&reconnect=auto' style='widt…

In [11]:
if is_plot:
    settings = {
        'BH': 'P6',
        'IH': 'P7',
    }

    for name, landmark in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_height(scene, df_local, file, landmark, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe3722d37f0_7&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe353716cd0_8&reconnect=auto' style='widt…

In [12]:
if is_plot:
    settings = {
        'BA': ['P4', 'P5', 'P8'],
        'T1A': ['P4', 'P2', 'P8'],
        'T5A': ['P5', 'P3', 'P9'],
    }

    for name, [landmark_origin, landmark1, landmark2] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_angle(scene, df_local, file, landmark_origin, landmark1, landmark2, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe3722be9d0_9&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe37116f880_10&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe35835ebe0_11&reconnect=auto' style='wid…

In [13]:
if is_plot:
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_circ_pass_landmarks(scene, df_local, file, mesh_local, ['P4', 'P5'], name='ABG')
    scene.show()

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe359407070_12&reconnect=auto' style='wid…

In [14]:
if is_plot:
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_circ_pass_landmark(scene, df_local, file, mesh_local, 'P6', 'x', name='IG')
    scene.show()

Widget(value="<iframe src='http://localhost:63468/index.html?ui=P_0x7fe359407670_13&reconnect=auto' style='wid…